# Immigration policy 

This is my first post. It includes a prototype of the data visualisation I intend to build with bokeh. The following updates will tackle the introduction of a slider and a dropdownmenu.

In [12]:
from bokeh.io import curdoc,push_notebook
from bokeh.models import ColumnDataSource, CategoricalColorMapper, Slider, Select
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row
import pandas as pd
import numpy as np 


MPI_WorldBank_DPI_cont=pd.read_csv('/Users/sergej/Google Drive/QASS_Master/Thesis_QASS/Data/MPI_WorldBank_DPI_cont.csv')
MPI_WorldBank_DPI_cont=MPI_WorldBank_DPI_cont.reset_index().rename(columns={'NY_GDP_PCAP_KD':'gdppc'})
MPI_WorldBank_DPI_cont.columns=[x.strip().replace(' ', '_') for x in MPI_WorldBank_DPI_cont.columns]
MPI_WorldBank_DPI_cont.loc[MPI_WorldBank_DPI_cont['Country_Code'].isin(['CAN','USA']),'continent']='NA'
data=MPI_WorldBank_DPI_cont[['gdppc','MPI_E','Country_Code','continent','MPI_S','MPI','MPI_I','SP_POP_DPND','SL_UEM_TOTL_ZS','SP_POP_TOTL','year']]
data.set_index(['year'])



# Create a HoverTool: hover
# Make a list of the unique values from the region column: regions_list
regions_list = data.continent.unique().tolist()

# Import CategoricalColorMapper from bokeh.models and the Spectral6 palette from bokeh.palettes

xmin, xmax = min(data.gdppc/1000), max(data.gdppc/1000)

# Save the minimum and maximum values of the life expectancy column: ymin, ymax
ymin, ymax = min(data.MPI_E), max(data.MPI_E)

data=data.set_index('year',drop=False)

source = ColumnDataSource( data = {
 'x'       : data.loc[1996].gdppc/1000,
 'y'       : data.loc[1996].MPI_E,
 'country' : data.loc[1996].Country_Code,
 'region' : data.loc[1996].continent,
 'pop'     : (data.loc[1996].SP_POP_TOTL / 20000000) + 2,
 'unemployment'  : data.loc[1996].SL_UEM_TOTL_ZS,
 'Immigration_Integration':data.loc[1996].MPI_I,
 'Immigration_Stay':data.loc[1996].MPI_S,
 'Immigration_Overall':data.loc[1996].MPI,
 'Age_dependency_ratio':data.loc[1996].SP_POP_DPND                   
  })



In [13]:
plot = figure(title='Immigration Policy for 1996', plot_height=400, plot_width=700,
              x_range=(xmin, xmax), y_range=(ymin, ymax))

# Add circle glyphs to the plot
plot.circle(x='x', y='y',  size='pop', fill_alpha=0.8, source=source)

# Set the x-axis label
plot.xaxis.axis_label ='GDP per Capita (const. thousand 2010 US $)'

# Set the y-axis label
plot.yaxis.axis_label = 'MPI (Openess)'




In [14]:
# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=regions_list, palette=Spectral6)

# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=0.8,  size='pop', source=source,
           color=dict(field='region', transform=color_mapper), legend='region')

# Set the legend.location attribute of the plot to 'top_right'
#plot.legend.location = 'top_right'

# Add the plot to the current document and add the title
#curdoc().add_root(plot)
#curdoc().title = 'Gapminder'

GlyphRenderer(id='1c96fcee-4e9e-42aa-a75b-ae4f013bc179', ...)

The code below already defines a callback, which allows to change the year displayed with a change of the slider. The code does not yet work in the jupyter notebook.

In [15]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # set the `yr` name to `slider.value` and `source.data = new_data`
    yr = slider.value
    new_data = {
        'x'       : data.loc[yr].gdppc/1000,
        'y'       : data.loc[yr].MPI_E,
        'country' : data.loc[yr].Country_Code,
        'region' : data.loc[yr].continent,
        'pop'     : (data.loc[yr].SP_POP_TOTL / 10^6),
        'unemployment'  : data.loc[yr].SL_UEM_TOTL_ZS,
        'Immigration_Integration':data.loc[yr].MPI_I,
        'Immigration_Stay':data.loc[yr].MPI_S,
        'Immigration_Overall':data.loc[yr].MPI,
        'Age_dependency_ratio':data.loc[yr].SP_POP_DPND                   
    }
    source.data = new_data
    push_notebook()


    # Add title to figure: plot.title.text
    plot.title.text = 'Immigration policy data for %d' % yr

# Make a slider object: slider
slider = Slider(start=1996,end=2012,step=1,value=1996,title='Year')

# Attach the callback to the 'value' property of slider
slider.on_change('value',update_plot)

from bokeh.models import HoverTool

# Create a HoverTool: hover
hover = HoverTool(tooltips=[('Country', '@country'),('Population (Mio.)','@pop'),('Unemployment','@unemployment')])

# Add the HoverTool to the plot
plot.add_tools(hover)

# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = row(widgetbox(slider), plot)
curdoc().add_root(layout)

In [16]:
output_notebook()
show(plot, notebook_handle=True)

Loading BokehJS ...